# Generate daily prices using daily data service - Python

### Overview
Daily prices is designed to illustrate price movements for an instrument over a look back period.

This sample demonstrates how to directly request and plot daily prices.<br>

It enables to retrieve **on-demand** daily prices data by calling a dedicated service on a `daily data store`. 

### Inputs/outputs
Daily prices sample requires instrument's identifier as per input. It returns daily prices and volumes.
This sample also shows how to plot a simple graph for basis financial analysis using an open source library.

### Services used
This sample uses *gRPC requests* in order to retrieve daily bars from the dedicated hosted service. The queried endpoint in this script are:
* *DailyPricesService*: to directly retrieve daily price objects from the server

### Modules required
1. Systemathics:
    * *systemathics.apis.type.shared.v1*
    * *systemathics.apis.services.daily.v1*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* as per display package

***

# Run daily prices sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas plotly

In [ ]:
pip install -i https://test.pypi.org/simple/ systemathics.apis==0.9.27

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.services.daily.v1.daily_prices_pb2 as daily_prices
import systemathics.apis.services.daily.v1.daily_prices_pb2_grpc as daily_prices_service

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Create and process request

#### 3.1 Request Parameters
To request *daily prices* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Service called: *DailyPricesService* in order to retrieve daily data

In [ ]:
# set the instrument
ticker = 'AAPL'
exchange = 'XNGS'

#### 3.2 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [ ]:
# create the daily prices request
request = daily_prices.DailyPricesRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker)
)

In [ ]:
# open a gRPC channel
credentials = grpc.ssl_channel_credentials()
with grpc.secure_channel("apis.systemathics.cloud:443", credentials) as channel:
    
    # instantiate the daily prices service
    service = daily_prices_service.DailyPricesServiceStub(channel)
    
    # process the daily prices request
    response = service.DailyPrices(
        request = request, 
        metadata = [('authorization', token)]
    )
    
print("Total daily prices retrieved: ",len(response.data))

### Step 4: Visualize data

#### 4.1 Retrieve daily prices
In the following code snippets, the reply is reprocessed in a dataframe in order to visualize the results:

In [ ]:
# prepare the dataframe content
dates=[datetime(p.date.year, p.date.month, p.date.day) for p in response.data]
prices = [p.price for p in response.data]
volumes = [p.volume for p in response.data]

d = {'Date': dates, 'Price': prices, 'Volume': volumes }

In [ ]:
# create pandas dataframe
df = pd.DataFrame(data=d)

In [ ]:
# visualize dataframe
df

#### 4.2 Plot daily prices
The following code snippets display daily prices' request results with the package of your choice.<br>*`plotly`* is used as per open source display package.

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=df['Date'], y=df['Price'], name='Price'), secondary_y=True)
fig.add_trace(go.Bar(x=df['Date'], y=df['Volume'], name="Volume"), secondary_y=False)

fig.update_xaxes(title_text = 'Date')
fig.update_layout(title = 'Daily prices for {0}-{1}'.format(ticker,exchange))
fig.layout.yaxis2.showgrid=False
fig.show()